In [1]:
import sys
import requests
from dotenv import load_dotenv
from crypto import convertDataToJSON, pinJSONtoIPFS, initContract, w3, pinFiletoIPFS

In [2]:
smart_contract = initContract()

Contract Address 0xd24bad916A6F6FA81D8258d6a7bca6545AC1496c


In [3]:
def registerArtwork():
    title = input("Enter a title for the artwork: ")
    description = input("Enter a description for the artwork: ")
    artists = input("Enter a comma seperated list of artists:")
    file_ipfs = input("Enter Image IPFS link:")
    
    message_object = {
            "title": title,
            "description": description,
            "artists": artists,
            "file_ipfs": file_ipfs,
    }

    return message_object

In [4]:
def postArtwork():
    message_object = registerArtwork() # Create New Message Object
    json_data = convertDataToJSON(message_object) # Convert message object to JSON
    message_uri = pinJSONtoIPFS(json_data) # Pin JSON to IPFS and get Hash
    
    tx_hash = smart_contract.functions.registerArt(str(message_uri)).transact(
        {"from": w3.eth.accounts[0]}
    )
    #logs = w3.eth.get_logs()
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)   
    return receipt #logs

In [5]:
def getLatestArtwork():
    ipfs_hash = smart_contract.functions.bid(token_id).call()
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json()

In [6]:
def getArtworkByID(message_id):
    ipfs_hash = smart_contract.functions.highestBid(message_id).call()
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json()

In [7]:
option = input("Would you like to get or post Artwork?")

if option == "post":
    postArtwork()

if option == "get": #get not functional refer to pullUri function
    token_id = input("Enter artwork ID to retrieve the artwork. Hit enter to get the latest.")

    if (token_id):
        print(getArtworkByID(int(token_id)))
    else:
        print(getLatestArtwork())

Would you like to get or post Artwork? post
Enter a title for the artwork:  the Origin
Enter a description for the artwork:  Pretty Colors
Enter a comma seperated list of artists: Kris
Enter Image IPFS link: bafybeichx2vhuc4645v6iesdq7ztz4664bi3x3r5mzvl3sm7qwyjr26z74


bafkreigjfoeiapyimiou3kmtndy4ozdfuz3tlcceqtfdklc5pa7xpnkihm


In [8]:
def auctionCreate(token_id):
    tx_hash = smart_contract.functions.createAuction(token_id).transact(
        {"from": w3.eth.accounts[0]}
    )
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)   #waitFor
    return receipt #logs

In [9]:
auctionCreate(1)

AttributeDict({'transactionHash': HexBytes('0xebb91501840c645b7612f23ba4ad07d293abd9b2cb9568d654687e7658410519'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x7438d913b4595d70fe4b34ac7545e308f8212bd5ba6fad1fa6ca325006476e1d'),
 'blockNumber': 70,
 'from': '0xbA58F51e201390af6F13AcA36b09c58A59Df7006',
 'to': '0xd24bad916A6F6FA81D8258d6a7bca6545AC1496c',
 'gasUsed': 562395,
 'cumulativeGasUsed': 562395,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [10]:
def placeBid(token_id):
    tx_hash = smart_contract.functions.bid(token_id).transact(
    {"from": w3.eth.accounts[0],
     'value': 100000 #Must hardcode bid value
    })
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [11]:
placeBid(1)

AttributeDict({'transactionHash': HexBytes('0x5459a743d917bed830fc4558ed49e9a6e08099d11fa17aef72adbd6b383b4d16'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x8eb8c5c04e24faaa6e0dc9b77bbc68954d1bf7207cccca6ea12864fbc64978e5'),
 'blockNumber': 71,
 'from': '0xbA58F51e201390af6F13AcA36b09c58A59Df7006',
 'to': '0xd24bad916A6F6FA81D8258d6a7bca6545AC1496c',
 'gasUsed': 76693,
 'cumulativeGasUsed': 76693,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x5459a743d917bed830fc4558ed49e9a6e08099d11fa17aef72adbd6b383b4d16'),
   'blockHash': HexBytes('0x8eb8c5c04e24faaa6e0dc9b77bbc68954d1bf7207cccca6ea12864fbc64978e5'),
   'blockNumber': 71,
   'address': '0x6F3E9f5C9e731BA2fb7AAE92a684b574D05045b9',
   'data': '0x000000000000000000000000ba58f51e201390af6f13aca36b09c58a59df700600000000000000000000000000000000000000000000000000000000000186a0',
   'topics': [HexBytes('0xf4757a49b326036464bec6fe419a4ae38c8a02ce3e68bf080

In [12]:
def closeAuction(token_id):
    tx_hash = smart_contract.functions.endAuction(token_id).transact({"from": w3.eth.accounts[0]})
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [15]:
closeAuction(1)

ContractLogicError: execution reverted: VM Exception while processing transaction: revert auctionEnd has already been called.

In [16]:
# Not functional due to openZeppelin having multiple safeTransferFrom
#def transfer(token_id):
   # tx_hash = smart_contract.functions.safeTransferFrom(token_id).transact(
    #{"from": w3.eth.accounts[0],
     #'to': w3.eth.accounts[1]
#Must hardcode bid value
    #})
    #receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    #return receipt

In [17]:
def pullUri(token_id):
    tx_hash = smart_contract.functions.tokenURI(token_id).call()
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt
    

In [18]:
pullUri(1)

ValueError: when sending a str, it must be a hex string. Got: 'bafkreidu52omlwpc22dzdf24ihurtwnms3rgdyzltapxvzg6wglkspyz6u'